In [1]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *

Loading ngbem library


         version v6.2.2305-13-g790f4784 but
         version v6.2.2305-16-g69025e5e is loaded at run-time!!!


In [2]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)
# mesh = Mesh(unit_cube.GenerateMesh(maxh=0.3))

In [3]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

In [4]:
fes = SurfaceL2(mesh, order=2, dual_mapping=True)
u,v = fes.TnT()
a = BilinearForm(fes)
SingleLayerPotential(a, intorder=16)
a.Assemble();

In [5]:
f = LinearForm (u0*v.Trace()*ds(bonus_intorder=3)).Assemble()

gfv = GridFunction(fes)
inv = a.mat.Inverse()
gfv.vec.data = inv * f.vec

In [6]:
Draw (gfv);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [7]:
gradu0 = CF( (u0.Diff(x), u0.Diff(y), u0.Diff(z)) )
n = specialcf.normal(3)
u1 = gradu0*n
Draw (u1, mesh, draw_vol=False);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [8]:
for t in Timers():
    if "SingleLayer" in t["name"]:
        print (t)

{'name': 'SingleLayer - disjoint2', 'time': 24.55448671959417, 'counts': 90910, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SingleLayer - disjoint', 'time': 26.61041194898356, 'counts': 90910, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SingleLayer - common edge', 'time': 11.34377154423237, 'counts': 924, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SingleLayer - common vertex', 'time': 13.380266964061242, 'counts': 2722, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SingleLayer - identic panel', 'time': 4.530227602815268, 'counts': 308, 'flops': 0.0, 'Gflop/s': 0.0}
{'name': 'SingleLayer - all', 'time': 56.01938799395096, 'counts': 1, 'flops': 0.0, 'Gflop/s': 0.0}


In [9]:
fesh1 = H1(mesh, order=3)
uH1,vH1 = fesh1.TnT()
b = BilinearForm(trialspace=fesh1, testspace=fes)
DoubleLayerPotential(b, intorder=16)
b += - 0.5 * uH1 * v.Trace() * ds(bonus_intorder=3)
b.Assemble();

In [10]:
gfdir = GridFunction(fesh1)
gfdir.Interpolate (u0)

gfv.vec.data = inv @ b.mat * gfdir.vec

In [ ]:
Draw (-gfv, mesh, draw_vol=False);